# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it using a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [1]:
# imports:
from itertools import combinations
from heapq import heappush, heappop

In [2]:
# Task 1:
NUM_COUPLES = 3
HUSBANDS = {f"H{i}" for i in range(1, NUM_COUPLES + 1)}
WIVES = {f"W{i}" for i in range(1, NUM_COUPLES + 1)}
EVERYONE = HUSBANDS | WIVES
BOAT_POS = ["left", "island", "right"]

class State:
    def __init__(self, left, island, right, boat_pos):
        self.left = left
        self.island = island
        self.right = right
        self.boat_pos = boat_pos

    def __repr__(self):
        return f"State(L={self.left}, I={self.island}, R={self.right}, boat={self.boat_pos})"

    def is_goal(self):
        return len(self.right) == len(EVERYONE) and self.boat_pos == "right"

    def copy(self):
        return State(
            left=self.left.copy(),
            island=self.island.copy(),
            right=self.right.copy(),
            boat_pos=self.boat_pos
        )

    def __eq__(self, other):
        return (
            isinstance(other, State) and
            self.left == other.left and
            self.island == other.island and
            self.right == other.right and
            self.boat_pos == other.boat_pos
        )

    def __hash__(self):
        return hash((frozenset(self.left), frozenset(self.island), frozenset(self.right), self.boat_pos))
    
    def is_valid(self, boat_passengers):
        return (
            is_group_valid(self.left) and
            is_group_valid(self.island) and
            is_group_valid(self.right) and
            is_group_valid(boat_passengers)
        )
    
    def get_adjacent_boat_positions(self):
        if self.boat_pos == "left" or self.boat_pos == "right":
            return ["island"]
        elif self.boat_pos == "island":
            return ["left", "right"]
        return []
    
    def generate_children(self):
        possible_passengers = []
        for i in range(1, 3):
            possible_passengers.extend(combinations(getattr(self, self.boat_pos), i))

        children = []
        for passengers in possible_passengers:
            passengers = set(passengers)
            for destination in self.get_adjacent_boat_positions():
                new_state = self.copy()
                getattr(new_state, self.boat_pos).difference_update(passengers)
                getattr(new_state, destination).update(passengers)
                new_state.boat_pos = destination
                if new_state.is_valid(passengers):
                    children.append((new_state, passengers))

        return children

def is_group_valid(group):
    for i in range(1, NUM_COUPLES + 1):
        wife = f"W{i}"
        husband = f"H{i}"
        other_husbands = {p for p in group if p.startswith("H") and p != husband}
        if wife in group and husband not in group and other_husbands:
            return False
    return True

initial_state = State(
    left=EVERYONE,
    island=set(),
    right=set(),
    boat_pos="left"
)

In [3]:
# Task 2:    
def dfs(initial_state: State):
    stack = [(initial_state, [])]
    visited = set()

    while stack:
        current_state, path = stack.pop()

        if current_state in visited:
            continue
        visited.add(current_state)

        if current_state.is_goal():
            print("Goal reached!\n")
            print(f"Initial state:\n{initial_state}\n")
            for step_num, (state, move) in enumerate(path, 1):
                print(f"Step {step_num}: Move {move} - {state}")
            print(f"\nFinal state:\n{current_state}")
            return 

        for child_state, passengers in current_state.generate_children():
            stack.append((child_state, path + [(child_state, passengers)]))

    print("No solution found.")

dfs(initial_state)

Goal reached!

Initial state:
State(L={'H2', 'W3', 'W1', 'H1', 'W2', 'H3'}, I=set(), R=set(), boat=left)

Step 1: Move {'W1', 'W2'} - State(L={'H2', 'W3', 'H1', 'H3'}, I={'W1', 'W2'}, R=set(), boat=island)
Step 2: Move {'W1', 'W2'} - State(L={'H2', 'W3', 'H1', 'H3'}, I=set(), R={'W1', 'W2'}, boat=right)
Step 3: Move {'W2'} - State(L={'H2', 'W3', 'H1', 'H3'}, I={'W2'}, R={'W1'}, boat=island)
Step 4: Move {'W2'} - State(L={'H2', 'W3', 'H1', 'W2', 'H3'}, I=set(), R={'W1'}, boat=left)
Step 5: Move {'W3', 'H3'} - State(L={'H2', 'H1', 'W2'}, I={'W3', 'H3'}, R={'W1'}, boat=island)
Step 6: Move {'H3'} - State(L={'H2', 'H1', 'W2', 'H3'}, I={'W3'}, R={'W1'}, boat=left)
Step 7: Move {'H1', 'H3'} - State(L={'H2', 'W2'}, I={'W3', 'H1', 'H3'}, R={'W1'}, boat=island)
Step 8: Move {'H1', 'H3'} - State(L={'H2', 'W2'}, I={'W3'}, R={'W1', 'H1', 'H3'}, boat=right)
Step 9: Move {'H3'} - State(L={'H2', 'W2'}, I={'W3', 'H3'}, R={'W1', 'H1'}, boat=island)
Step 10: Move {'W3', 'H3'} - State(L={'H2', 'W2'}, I=s

In [4]:
# Task 3:
def backtrack(state, path, visited):
    if state.is_goal():
        return path

    visited.add(state)

    for child_state, passengers in state.generate_children():
        if child_state not in visited:
            result = backtrack(child_state, path + [(child_state, passengers)], visited)
            if result:
                return result

    return None


solution = backtrack(initial_state, [(initial_state, None)], set())

if solution:
    print("Goal reached!\n")
    print(f"Initial state:\n{initial_state}\n")
    for step_num, (state, move) in enumerate(solution, 1):
        if move is not None:
            print(f"Step {step_num - 1}: Move {move} - {state}")
    print(f"\nFinal state:\n{state}")
else:
    print("No solution found.")

Goal reached!

Initial state:
State(L={'H2', 'W3', 'W1', 'H1', 'W2', 'H3'}, I=set(), R=set(), boat=left)

Step 1: Move {'H2', 'W2'} - State(L={'W3', 'W1', 'H1', 'H3'}, I={'H2', 'W2'}, R=set(), boat=island)
Step 2: Move {'H2'} - State(L={'H2', 'W3', 'W1', 'H1', 'H3'}, I={'W2'}, R=set(), boat=left)
Step 3: Move {'W3'} - State(L={'H2', 'W1', 'H1', 'H3'}, I={'W3', 'W2'}, R=set(), boat=island)
Step 4: Move {'W2'} - State(L={'H2', 'W1', 'H1', 'W2', 'H3'}, I={'W3'}, R=set(), boat=left)
Step 5: Move {'W1'} - State(L={'H2', 'H1', 'W2', 'H3'}, I={'W1', 'W3'}, R=set(), boat=island)
Step 6: Move {'W3'} - State(L={'H2', 'W3', 'H1', 'W2', 'H3'}, I={'W1'}, R=set(), boat=left)
Step 7: Move {'H1'} - State(L={'H2', 'W3', 'W2', 'H3'}, I={'W1', 'H1'}, R=set(), boat=island)
Step 8: Move {'W1', 'H1'} - State(L={'H2', 'W3', 'W2', 'H3'}, I=set(), R={'W1', 'H1'}, boat=right)
Step 9: Move {'H1'} - State(L={'H2', 'W3', 'W2', 'H3'}, I={'H1'}, R={'W1'}, boat=island)
Step 10: Move {'H1'} - State(L={'H2', 'W3', 'H1'

In [5]:
# Task 4:
def heuristic(state: State):
    return len(EVERYONE - state.right)

class Node:
    def __init__(self, state: State, parent, cost, move):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.move = move

    def __lt__(self, other):
        return (self.cost + heuristic(self.state)) < (other.cost + heuristic(other.state))

def reconstruct_path(node: Node):
    path = []
    while node:
        path.append((node.state, node.move))
        node = node.parent
    return list(reversed(path))

def a_star_search(initial_state: State):
    heap_queue = []
    heappush(heap_queue, Node(initial_state, None, 0, None))
    visited: Dict[State, int] = {initial_state: 0}

    while heap_queue:
        current_node = heappop(heap_queue)
        current_state = current_node.state

        if current_state.is_goal():
            return reconstruct_path(current_node)

        for child_state, move in current_state.generate_children():
            new_cost = current_node.cost + 1
            if child_state not in visited or new_cost < visited[child_state]:
                visited[child_state] = new_cost
                heappush(heap_queue, Node(child_state, current_node, new_cost, move))

    return None

sol = a_star_search(initial_state)

if sol:
    print("Goal reached!\n")
    print(f"Initial state:\n{sol[0][0]}\n")
    for step_num, (state, move) in enumerate(sol[1:], 1):
        move_str = ', '.join(sorted(move)) if move else "None"
        print(f"Step {step_num}: Move {{{move_str}}} - {state}")
    print(f"\nFinal state:\n{sol[-1][0]}")
else:
    print("No solution found.")

Goal reached!

Initial state:
State(L={'H2', 'W3', 'W1', 'H1', 'W2', 'H3'}, I=set(), R=set(), boat=left)

Step 1: Move {W1, W2} - State(L={'H2', 'W3', 'H1', 'H3'}, I={'W1', 'W2'}, R=set(), boat=island)
Step 2: Move {W2} - State(L={'H2', 'W3', 'H1', 'W2', 'H3'}, I={'W1'}, R=set(), boat=left)
Step 3: Move {W2, W3} - State(L={'H2', 'H1', 'H3'}, I={'W1', 'W3', 'W2'}, R=set(), boat=island)
Step 4: Move {W2} - State(L={'H2', 'H1', 'W2', 'H3'}, I={'W1', 'W3'}, R=set(), boat=left)
Step 5: Move {H1, H3} - State(L={'H2', 'W2'}, I={'W1', 'W3', 'H1', 'H3'}, R=set(), boat=island)
Step 6: Move {W1, W3} - State(L={'H2', 'W2'}, I={'H1', 'H3'}, R={'W1', 'W3'}, boat=right)
Step 7: Move {W3} - State(L={'H2', 'W2'}, I={'W3', 'H1', 'H3'}, R={'W1'}, boat=island)
Step 8: Move {H1} - State(L={'H2', 'H1', 'W2'}, I={'W3', 'H3'}, R={'W1'}, boat=left)
Step 9: Move {H1, H2} - State(L={'W2'}, I={'H2', 'W3', 'H1', 'H3'}, R={'W1'}, boat=island)
Step 10: Move {H1, H2} - State(L={'W2'}, I={'W3', 'H3'}, R={'W1', 'H2', '